In [2]:
import pandas as pd
import json
import numpy as np

In [107]:
with open ('1384403.json') as f:
    data = json.load(f)

In [108]:
data

{'meta': {'data_version': '1.1.0', 'created': '2023-10-20', 'revision': 1},
 'info': {'balls_per_over': 6,
  'city': 'Ahmedabad',
  'dates': ['2023-10-14'],
  'event': {'name': 'ICC Cricket World Cup', 'match_number': 12},
  'gender': 'male',
  'match_type': 'ODI',
  'match_type_number': 4669,
  'officials': {'match_referees': ['AJ Pycroft'],
   'reserve_umpires': ['CB Gaffaney'],
   'tv_umpires': ['RA Kettleborough'],
   'umpires': ['M Erasmus', 'RK Illingworth']},
  'outcome': {'winner': 'India', 'by': {'wickets': 7}},
  'overs': 50,
  'player_of_match': ['JJ Bumrah'],
  'players': {'Pakistan': ['Abdullah Shafique',
    'Imam-ul-Haq',
    'Babar Azam',
    'Mohammad Rizwan',
    'Saud Shakeel',
    'Iftikhar Ahmed',
    'Shadab Khan',
    'Mohammad Nawaz',
    'Hasan Ali',
    'Shaheen Shah Afridi',
    'Haris Rauf'],
   'India': ['RG Sharma',
    'Shubman Gill',
    'V Kohli',
    'SS Iyer',
    'KL Rahul',
    'HH Pandya',
    'RA Jadeja',
    'SN Thakur',
    'JJ Bumrah',
    'Kul

In [101]:
data.keys()

dict_keys(['meta', 'info', 'innings'])

In [109]:
data["innings"][1]["overs"]

[{'over': 0,
  'deliveries': [{'batter': 'RG Sharma',
    'bowler': 'Shaheen Shah Afridi',
    'non_striker': 'Shubman Gill',
    'runs': {'batter': 4, 'extras': 0, 'total': 4}},
   {'batter': 'RG Sharma',
    'bowler': 'Shaheen Shah Afridi',
    'non_striker': 'Shubman Gill',
    'runs': {'batter': 1, 'extras': 0, 'total': 1}},
   {'batter': 'Shubman Gill',
    'bowler': 'Shaheen Shah Afridi',
    'non_striker': 'RG Sharma',
    'runs': {'batter': 4, 'extras': 0, 'total': 4}},
   {'batter': 'Shubman Gill',
    'bowler': 'Shaheen Shah Afridi',
    'non_striker': 'RG Sharma',
    'runs': {'batter': 0, 'extras': 0, 'total': 0}},
   {'batter': 'Shubman Gill',
    'bowler': 'Shaheen Shah Afridi',
    'extras': {'legbyes': 1},
    'non_striker': 'RG Sharma',
    'runs': {'batter': 0, 'extras': 1, 'total': 1}},
   {'batter': 'RG Sharma',
    'bowler': 'Shaheen Shah Afridi',
    'non_striker': 'Shubman Gill',
    'runs': {'batter': 0, 'extras': 0, 'total': 0}}]},
 {'over': 1,
  'deliveries': 

In [35]:
over_list = []
batter_list = []
bowler_list = []
non_striker_list = []
batter_runs_list = []
extras_list = []
total_runs_list = []

In [56]:
# Extract the Pakistan innings data
innings_data = data["innings"][0]["overs"]

# Create lists to store the data
over_list = []
batter_list = []
bowler_list = []
non_striker_list = []
batter_runs_list = []
extras_list = []
total_runs_list = []
wickets_list = []

# Iterate through the innings data to extract required information
for over in innings_data:
    for delivery in over["deliveries"]:
        over_list.append(over["over"])
        batter_list.append(delivery["batter"])
        bowler_list.append(delivery["bowler"])
        non_striker_list.append(delivery["non_striker"])
        batter_runs_list.append(delivery["runs"]["batter"])
        extras_list.append(delivery["runs"]["extras"])
        total_runs_list.append(delivery["runs"]["total"])
        if "wickets" in delivery:
            wickets_list.append(delivery["wickets"][0]["player_out"] + " (" + delivery["wickets"][0]["kind"] + ")")
        else:
            wickets_list.append(None)

# Create a DataFrame from the extracted data
df_pakistan_innings = pd.DataFrame({
    "Over": over_list,
    "Batter": batter_list,
    "Bowler": bowler_list,
    "Non_Striker": non_striker_list,
    "Runs": batter_runs_list,
    "Extras": extras_list,
    "Total_Runs": total_runs_list,
    "Wicket": wickets_list
})

# Display the DataFrame
df_pakistan_innings.head()

,Over,Batter,Bowler,Non_Striker,Runs,Extras,Total_Runs,Wicket
0,0,Abdullah Shafique,JJ Bumrah,Imam-ul-Haq,0,0,0,None
1,0,Abdullah Shafique,JJ Bumrah,Imam-ul-Haq,0,0,0,None
2,0,Abdullah Shafique,JJ Bumrah,Imam-ul-Haq,0,0,0,None
3,0,Abdullah Shafique,JJ Bumrah,Imam-ul-Haq,0,0,0,None
4,0,Abdullah Shafique,JJ Bumrah,Imam-ul-Haq,0,0,0,None


In [116]:
# Define the Pandas UDF
def get_player_stats(df, batsman, bowler):
    filtered_df = df[(df['Batter'] == batsman) & (df['Bowler'] == bowler)]
    runs_scored = filtered_df['Runs'].sum()
    balls_bowled = filtered_df.shape[0]
    wickets_taken = filtered_df.loc[df['Wicket'].notnull(), 'Wicket'].count()
    strike_rate = (runs_scored / balls_bowled) * 100
    dot_balls =  filtered_df.loc[(df['Runs'] == 0), 'Runs'].count()
    fours = filtered_df.loc[(df['Runs'] == 4), 'Runs'].count()
    sixes = filtered_df.loc[(df['Runs'] == 6), 'Runs'].count()
    dot_percentage = (dot_balls/balls_bowled) * 100
    boundary_percentage = ((fours + sixes)/balls_bowled) * 100
    
    stats = {
        "runs": runs_scored,
        "balls": balls_bowled,
        "wickets": wickets_taken,
        "dots": dot_balls,
        "strike_rate": strike_rate,
        "4s": fours,
        "6s": sixes,
        "dot%": dot_percentage,
        "boundary%": boundary_percentage}
    return stats

In [88]:
# Call the UDF with desired parameters
batsman_name = "Mohammad Rizwan"
bowler_name = "Kuldeep Yadav"
stas1 = get_player_stats(df_pakistan_innings, batsman_name, bowler_name)

# Display the results
stas1

{'runs': 9,
 'balls': 18,
 'wickets': 0,
 'dots': 12,
 'strike_rate': 50.0,
 '4s': 1,
 '6s': 0,
 'dot%': 66.66666666666666}

In [1]:
esult = np.divide(numerator, denominator, out=np.zeros_like(numerator), where=denominator != 0)

NameError: name 'np' is not defined

In [ ]:
RuntimeWarning: divide by zero encountered in longlong_scalars

In [110]:
# Function to convert innings data to DataFrame
def create_pandas_df(innings_data):
    # Create lists to store the data
    over_list = []
    batter_list = []
    bowler_list = []
    non_striker_list = []
    batter_runs_list = []
    extras_list = []
    total_runs_list = []
    wickets_list = []

    # Iterate through the innings data to extract required information
    for over in innings_data:
        for delivery in over["deliveries"]:
            over_list.append(over["over"])
            batter_list.append(delivery["batter"])
            bowler_list.append(delivery["bowler"])
            non_striker_list.append(delivery["non_striker"])
            batter_runs_list.append(delivery["runs"]["batter"])
            extras_list.append(delivery["runs"]["extras"])
            total_runs_list.append(delivery["runs"]["total"])
            if "wickets" in delivery:
                wickets_list.append(delivery["wickets"][0]["player_out"] + " (" + delivery["wickets"][0]["kind"] + ")")
            else:
                wickets_list.append(None)

    # Create a DataFrame from the extracted data
    df_pakistan_innings = pd.DataFrame({
        "Over": over_list,
        "Batter": batter_list,
        "Bowler": bowler_list,
        "Non_Striker": non_striker_list,
        "Runs": batter_runs_list,
        "Extras": extras_list,
        "Total_Runs": total_runs_list,
        "Wicket": wickets_list
    })

    return df_pakistan_innings

# Extract the Pakistan innings data
innings_data = data["innings"][1]["overs"]

# Convert innings data to DataFrame
df_india = create_pandas_df(innings_data)

# Display the DataFrame
df_india.head()

,Over,Batter,Bowler,Non_Striker,Runs,Extras,Total_Runs,Wicket
0,0,RG Sharma,Shaheen Shah Afridi,Shubman Gill,4,0,4,None
1,0,RG Sharma,Shaheen Shah Afridi,Shubman Gill,1,0,1,None
2,0,Shubman Gill,Shaheen Shah Afridi,RG Sharma,4,0,4,None
3,0,Shubman Gill,Shaheen Shah Afridi,RG Sharma,0,0,0,None
4,0,Shubman Gill,Shaheen Shah Afridi,RG Sharma,0,1,1,None


In [125]:
# Call the UDF with desired parameters
batsman_name = "Babar Azam"
bowler_name = "Kuldeep Yadav"
stas1 = get_player_stats(df_pakistan_innings, batsman_name, bowler_name)

# Display the results
stas1

{'runs': 10,
 'balls': 18,
 'wickets': 0,
 'dots': 11,
 'strike_rate': 55.55555555555556,
 '4s': 1,
 '6s': 0,
 'dot%': 61.111111111111114,
 'boundary%': 5.555555555555555}